In [1]:
#import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
from scipy.stats import linregress
import matplotlib.cm as cm
import matplotlib.colors as mcol

In [69]:
#Reading in csv and previewing data
movie_data_df = pd.read_csv("imdb (1000 movies) in june 2022.csv")
movie_data_df.head()

,ranking of movie\r\n,movie name\r\n,Year,certificate,runtime,genre,RATING,metascore,DETAIL ABOUT MOVIE\n,DIRECTOR\r\n,ACTOR 1\n,ACTOR 2\n,ACTOR 3,ACTOR 4,votes,GROSS COLLECTION\r\n
0,1,The Shawshank Redemption,-1994,15,142 min,Drama,9.3,81.0,Two imprisoned men bond over a number of years...,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,"2,603,314",$28.34M
1,2,The Godfather,-1972,X,175 min,"Crime, Drama",9.2,100.0,The aging patriarch of an organized crime dyna...,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,"1,798,731",$134.97M
2,3,The Dark Knight,-2008,12A,152 min,"Action, Crime, Drama",9.0,84.0,When the menace known as the Joker wreaks havo...,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,"2,574,810",$534.86M
3,4,The Lord of the Rings: The Return of the King,-2003,12A,201 min,"Action, Adventure, Drama",9.0,94.0,Gandalf and Aragorn lead the World of Men agai...,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,"1,787,701",$377.85M
4,5,Schindler's List,-1993,15,195 min,"Biography, Drama, History",9.0,94.0,"In German-occupied Poland during World War II,...",Steven Spielberg,Liam Neeson,Ralph Fiennes,Ben Kingsley,Caroline Goodall,"1,323,776",$96.90M


In [138]:
#renaming columns
clean_df = movie_data_df.rename(columns={
    "ranking of movie\r\n":"IMDB Rank",
    "movie name\r\n":"Movie Name",
    "certificate":"Rating",
    "runtime":"Runtime (Minutes)",
    "genre":"Genre",
    "RATING":"IMDB Score",
    "metascore":"Metascore",
    "DETAIL ABOUT MOVIE\n":"Details",
    "DIRECTOR\r\n":"Director",
    "ACTOR 1\n":"Actor 1",
    "ACTOR 2\n": "Actor 2",
    "ACTOR 3":"Actor 3",
    "ACTOR 4":"Actor 4",
    "votes":"IMDB Votes",
    "GROSS COLLECTION\r\n":"BoxOffice"
})


#cleaning up columns and setting data types
#dropping null values
clean_df=clean_df.dropna()

# #cleaning year - removing "-" and setting to int
# clean_df["Year"]=clean_df["Year"].str.split("-").str[1]
# clean_df=clean_df.dropna()
clean_df['Year'] = clean_df['Year'].str.extract(pat='(\d+)', expand=False)
clean_df["Year"]=clean_df["Year"].astype("int")

#cleaning runtime and setting as int
clean_df["Runtime (Minutes)"]=clean_df["Runtime (Minutes)"].str.split(" ").str[0]
clean_df["Runtime (Minutes)"]=clean_df["Runtime (Minutes)"].astype("int")

#clean up Genre to only get the first genre
clean_df["Genre"]=clean_df["Genre"].str.split(",").str[0]

#setting IMDB Score as int
clean_df["IMDB Score"]=clean_df["IMDB Score"].astype("float")

#cleaning box office. Removing "$" and "M" and setting to float
clean_df["BoxOffice"]=clean_df["BoxOffice"].str.split("M").str[0]
clean_df["BoxOffice"]=clean_df["BoxOffice"].str.split("$").str[1]
clean_df["BoxOffice"]=clean_df["BoxOffice"].astype("float")

#Cleaning up IMDB votes and setting as int
clean_df["IMDB Votes"]=clean_df["IMDB Votes"].str.replace(",","")
clean_df["IMDB Votes"]=clean_df["IMDB Votes"].astype("int")

#resetting index and displaying
clean_df=clean_df.reset_index()
clean_df.info()






<class 'pandas.core.frame.DataFrame'>
RangeIndex: 738 entries, 0 to 737
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              738 non-null    int64  
 1   IMDB Rank          738 non-null    object 
 2   Movie Name         738 non-null    object 
 3   Year               738 non-null    int32  
 4   Rating             738 non-null    object 
 5   Runtime (Minutes)  738 non-null    int32  
 6   Genre              738 non-null    object 
 7   IMDB Score         738 non-null    float64
 8   Metascore          738 non-null    float64
 9   Details            738 non-null    object 
 10  Director           738 non-null    object 
 11  Actor 1            738 non-null    object 
 12  Actor 2            738 non-null    object 
 13  Actor 3            738 non-null    object 
 14  Actor 4            738 non-null    object 
 15  IMDB Votes         738 non-null    int32  
 16  BoxOffice          738 non

In [126]:
clean_df.isnull().sum()

index                  0
IMDB Rank              0
Movie Name             0
Year                   0
Rating                 5
Runtime (Minutes)      0
Genre                  0
IMDB Score             0
Metascore            163
Details                0
Director               0
Actor 1                0
Actor 2                0
Actor 3                0
Actor 4                0
IMDB Votes             0
BoxOffice            180
dtype: int64